In [1]:
import pandas as pd
import numpy as np

In [2]:
RELATIVE_IN = "2. raw"
RELATIVE_OUT = "3. curated"

## Offers

In [3]:
offers_raw = pd.read_csv(f"{RELATIVE_IN}/offers.csv", index_col=0)
print(offers_raw.shape)
#print(offers_raw.dtypes)

(952, 31)


### Fixing Values

In [4]:
offers_raw["offer uni"].value_counts()

offer uni
The University of Melbourne                        163
Deakin University                                  104
Griffith University                                 84
The University of Western Australia                 61
The University of Notre Dame Sydney                 54
The University of Notre Dame Fremantle              51
Australian National University                      32
The University of Queensland (CQ-WB RMP)            30
The University of Queensland                        29
The University of Wollongong                        27
Macquarie University                                27
The University of Queensland (Greater Brisbane)     14
The University of Queensland (DD MP)                12
Name: count, dtype: int64

In [5]:
UQ_NAME = "The University of Queensland"
RMP_ENDINGS = ["(DD MP)", "(CQ-WB RMP)"]
METRO_ENDING = "(Greater Brisbane)"

# fixing the queensland column
offers = offers_raw.copy()

# get the RMP types
offers.loc[offers["uq type"] == "RMP", "offer uni"] = UQ_NAME + " (RMP)"

# change the MD and WB to correct types
offers.loc[offers["offer uni"].isin([f"{UQ_NAME} {x}" for x in RMP_ENDINGS ]), "offer uni"] = UQ_NAME + " (RMP)"
offers.loc[offers["interview uni"].isin([f"{UQ_NAME} {x}" for x in RMP_ENDINGS ]), "interview uni"] = UQ_NAME + " (RMP)"

# change greater brisbance
metro_mask_offer = (offers["offer uni"] == UQ_NAME) | (offers["offer uni"] == f"{UQ_NAME} {METRO_ENDING}")
metro_mask_interview = (offers["interview uni"] == UQ_NAME) | (offers["interview uni"] == f"{UQ_NAME} {METRO_ENDING}")
offers.loc[metro_mask_offer, "offer uni"] = f"{UQ_NAME} (Metro)"
offers.loc[metro_mask_interview, "interview uni"] = f"{UQ_NAME} (Metro)"

# dropping the type
offers.drop(columns="uq type", inplace=True)

# fixing the interview column
offers.loc[(offers["year"] == 2023) & (offers["interview uni"].isna()), "interviewed?"] = "Yes"
offers.loc[(offers["year"] == 2023) & (offers["interview uni"].notna() & (offers["offer uni"].notna())), "interviewed?"] = "No"

# make the notes lowe case
offers.loc[:, "notes"] = offers["notes"].str.lower()

# drop un necessary columns
offers.drop(columns=["status", "timestamp"], inplace=True)

In [6]:
offers["offer uni"].value_counts()

offer uni
The University of Melbourne               163
Deakin University                         104
Griffith University                        84
The University of Queensland (RMP)         61
The University of Western Australia        61
The University of Notre Dame Sydney        54
The University of Notre Dame Fremantle     51
Australian National University             32
The University of Wollongong               27
Macquarie University                       27
The University of Queensland (Metro)       24
Name: count, dtype: int64

### Getting the marker type column

In [7]:
print(offers[(offers["offer uni place type"].isna()) & (offers["offer uni"].isna())].shape)  # offer uni and offer uni type notna
print(offers[(offers["offer uni"].isna())].shape)                                            # offer uni notna
print(offers[(offers["offer uni place type"].isna())].shape)                                 # offer uni type notna

print(offers[(offers["offer uni"].isna()) & (offers["places selected"].isna())].shape)       # offer uni na with places selected also na

(264, 28)
(264, 28)
(264, 28)
(0, 28)


In [8]:
offers["places selected"].value_counts()

places selected
CSP, BMP          327
CSP, BMP, FFP     119
All                94
CSP & BMP Only     88
CSP                47
CSP Only           29
FFP                 5
BMP                 5
CSP, FFP            5
BMP, FFP            1
Name: count, dtype: int64

In [9]:
# fixing the places selected column with dictionary
CSP = "CSP"
BMP = "BMP"
FFP = "FFP"

rename_places_selected = {
    "CSP, BMP": [CSP, BMP],
    "CSP & BMP Only": [CSP, BMP],

    "CSP, BMP, FFP": [CSP, BMP, FFP],
    "All": [CSP, BMP, FFP],

    "CSP": [CSP],
    "CSP Only": [CSP],
    "CSP, FFP": [CSP, FFP],
    
    "BMP": [BMP],
    "BMP, FFP": [BMP, FFP],

    "FFP": [FFP],
}

offers["places selected"] = offers["places selected"].apply(lambda x: rename_places_selected[x] 
                                                            if (not pd.isna(x)) and (rename_places_selected.get(x)) else x)

offers["places selected"].value_counts()

places selected
[CSP, BMP]         415
[CSP, BMP, FFP]    213
[CSP]               76
[FFP]                5
[BMP]                5
[CSP, FFP]           5
[BMP, FFP]           1
Name: count, dtype: int64

In [10]:
offers['offer uni place type'].value_counts()

offer uni place type
CSP    491
BMP    131
FFP     66
Name: count, dtype: int64

In [11]:
# checks how many times someone gets rejected and the places they selected aren't recorded
offers[offers["places selected"].isna()]["interview uni"].notna().sum()

0

In [12]:
offers["marker"] = offers["offer uni place type"]
offers.loc[offers["marker"].isna(), "marker"] = offers["places selected"]
#offers.loc[offers["marker"].isna(), "marker"] = "Unknown"
offers["marker"].isna().sum()

#offers["marker"].value_counts()

0

### Get Numeric

In [13]:
# get discrete interview opinion

DISCRETIZE_INTERIVEW = {"Very poorly": 1, "Poorly": 2, "Unsure": 3, "Well": 4, "Very well": 5}

offers["interview opinion discrete"] = offers["interview opinion"].apply(lambda x: DISCRETIZE_INTERIVEW[x])

offers[["interview opinion", "interview opinion discrete"]].tail(5)

,interview opinion,interview opinion discrete
947,Well,4
948,Poorly,2
949,Very poorly,1
950,Well,4
951,Very well,5


In [14]:
# get discrete interview hours

def get_first_digit(x):
    if (pd.isna(x)):
        return x
    elif ("-" in x):
        return float(x.split("-")[0])
    else:
        return float(x.split("+")[0])

def apply_rank(series):
    # get the not na series values
    series_values = sorted([x for x in series.unique() if not pd.isna(x)])

    # get the dictionary for the rankings
    rank_dict = {value: index for index, value in enumerate(series_values)}

    # apply the rank dict
    series = series.apply(lambda x: rank_dict.get(x))

    return series

# get the first digit of each prep hour
offers["interview prep hours discrete"] = offers["interview prep hours"].apply(get_first_digit)

# get the rankings for each hour
offers.loc[:, "interview prep hours discrete"] = apply_rank(offers["interview prep hours discrete"])

offers[["interview prep hours", "interview prep hours discrete"]].tail(5)

,interview prep hours,interview prep hours discrete
947,26-50,3.0
948,51-100,4.0
949,26-50,3.0
950,101-250,5.0
951,26-50,3.0


In [15]:
'offer uni place type'
'places selected'

'places selected'

In [16]:
offers.columns

Index(['rurality', 's1 score', 's2 score', 's3 score', 'uw gamsat', 'w gamsat',
       'offer uni', 'offer uni gpa', 'offer uni gamsat',
       'offer uni place type', 'offer uni preference', 'interviewed?',
       'interview uni', 'interview uni gpa', 'interview uni gamsat',
       'places selected', 'interview uni preference', 'deakin bonus',
       'anu bonus', 'mq bonus (gpa)', 'casper quartile', 'uq rmp tier',
       'gemsas over other?', 'interview opinion', 'notes', 'year',
       'interview prep hours', 'other rejections', 'marker',
       'interview opinion discrete', 'interview prep hours discrete'],
      dtype='object')

### Save CSV

In [17]:
offers.to_csv(f"{RELATIVE_OUT}/offer.csv")

### Messing around

In [165]:
(offers["interview uni"] == UQ_NAME).sum()

67

In [159]:
def uq_mask(series):
    return series.apply(lambda x: UQ_NAME in x if not pd.isna(x) else False)

In [160]:
offers.apply(lambda x: x.count())

rurality                    952
s1 score                    952
s2 score                    952
s3 score                    952
uw gamsat                   952
w gamsat                    952
offer uni                   688
offer uni gpa               688
offer uni gamsat            688
offer uni place type        688
offer uni preference        688
interviewed?                688
interview uni               385
interview uni gpa           385
interview uni gamsat        385
places selected             720
interview uni preference    385
deakin bonus                952
anu bonus                   952
mq bonus (gpa)              952
casper quartile             417
uq rmp tier                 161
gemsas over other?          688
interview opinion           952
notes                       567
year                        952
interview prep hours        509
other rejections            142
dtype: int64

## Interviews

In [21]:
interview_raw = pd.read_csv(f"{RELATIVE_IN}/interview.csv", index_col=0)
print(interview_raw.shape)
print(interview_raw.dtypes)

(1533, 44)
timestamp                object
rurality                 object
s1 score                  int64
s2 score                  int64
s3 score                  int64
uw gamsat               float64
w gamsat                float64
deakin bonus              int64
anu bonus                 int64
mq bonus                  int64
notes                    object
interview?               object
interview uni            object
interview uni gpa       float64
interview uni gamsat    float64
pref 1 uni               object
pref 1 gpa              float64
pref 1 gamsat           float64
pref 2 uni               object
pref 2 gpa              float64
pref 2 gamsat           float64
pref 3 uni               object
pref 3 gpa              float64
pref 3 gamsat           float64
pref 4 uni               object
pref 4 gpa              float64
pref 4 gamsat           float64
pref 5 uni               object
pref 5 gpa              float64
pref 5 gamsat           float64
pref 6 uni               obje

### Changing UQ

In [19]:
interview_raw.columns

Index(['timestamp', 'rurality', 's1 score', 's2 score', 's3 score',
       'uw gamsat', 'w gamsat', 'deakin bonus', 'anu bonus', 'mq bonus',
       'notes', 'interview?', 'interview uni', 'interview uni gpa',
       'interview uni gamsat', 'pref 1 uni', 'pref 1 gpa', 'pref 1 gamsat',
       'pref 2 uni', 'pref 2 gpa', 'pref 2 gamsat', 'pref 3 uni', 'pref 3 gpa',
       'pref 3 gamsat', 'pref 4 uni', 'pref 4 gpa', 'pref 4 gamsat',
       'pref 5 uni', 'pref 5 gpa', 'pref 5 gamsat', 'pref 6 uni', 'pref 6 gpa',
       'pref 6 gamsat', 'duplicates?', 'year', 'uq tier', 'casper quartile',
       'outlier', 'deakin tier', 'unimelb gam', 'undf bonuses', 'unds bonuses',
       'location', 'uow bonuses'],
      dtype='object')

In [23]:
UNI_COLUMNS = ["interview uni"] + [f"pref {i+1} uni" for i in range(6)]
UQ_NAME = "The University of Queensland"
RMP_ENDINGS = ["(DD MP)", "(CQ-WB RMP)", "(RMP/DDMP)"]
METRO_ENDING = "(Greater Brisbane)"

# fixing the queensland column
interview = interview_raw.copy()

# change the MD and WB to correct types
for uni_column in UNI_COLUMNS:
    interview.loc[interview[uni_column].isin([f"{UQ_NAME} {x}" for x in RMP_ENDINGS ]), uni_column] = f"{UQ_NAME} (RMP)"
    interview.loc[interview[uni_column] == f"{UQ_NAME} {METRO_ENDING}", uni_column] = f"{UQ_NAME} (Metro)"

# make the notes lowe case
interview.loc[:, "notes"] = interview["notes"].str.lower()

In [25]:
#interview.groupby("interview uni")["year"].value_counts()
interview["interview uni"].value_counts()

interview uni
The University of Melbourne               300
The University of Notre Dame Sydney       198
Deakin University                         126
Griffith University                       110
The University of Western Australia       108
The University of Notre Dame Fremantle    101
The University of Wollongong               93
The University of Queensland (RMP)         83
The University of Queensland               74
Australian National University             62
The University of Queensland (Metro)       60
Macquarie University                       52
Name: count, dtype: int64

In [27]:
interview.to_csv(f"{RELATIVE_OUT}/interview.csv")